In [1]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'} # each region has its XGBoost container
my_region = boto3.session.Session().region_name # set the region of the instance
print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the us-east-2 region. You will use the 825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [3]:
bucket_name = 'firewhere0602'
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [8]:
data = pd.read_csv("data.csv")

In [9]:
#df.loc[(df.Event == 'Dance'),'Event']='Hip-Hop'
data.loc[(data.frp <= 5), 'frp'] = 0
data.loc[(data.frp > 5), 'frp'] = 1
data.head(10)

,latitude,longitude,frp
0,-37.48861,149.63156,0.0
1,-34.46110,150.88142,0.0
2,-33.94823,151.21292,1.0
3,-34.45618,150.87723,0.0
4,-31.60223,150.15147,1.0
5,-29.26867,149.46602,1.0
6,-29.26898,149.46230,1.0
7,-31.60248,150.14816,1.0
8,-29.26928,149.45857,0.0
9,-29.34156,148.04869,1.0


In [11]:
train_data, test_data = np.split(data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(73999, 3) (31714, 3)


In [13]:
pd.concat([train_data['frp'], train_data.drop('frp', axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [14]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(containers[my_region],role, train_instance_count=1, train_instance_type='ml.m4.xlarge',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [15]:
xgb.fit({'train': s3_input_train})

2021-06-01 18:48:28 Starting - Starting the training job...
2021-06-01 18:48:30 Starting - Launching requested ML instancesProfilerReport-1622573308: InProgress
......
2021-06-01 18:49:43 Starting - Preparing the instances for training......
2021-06-01 18:50:46 Downloading - Downloading input data...
2021-06-01 18:51:23 Training - Downloading the training image..Arguments: train
[2021-06-01:18:51:35:INFO] Running standalone xgboost training.
[2021-06-01:18:51:35:INFO] Path /opt/ml/input/data/validation does not exist!
[2021-06-01:18:51:35:INFO] File size need to be processed in the node: 2.03mb. Available memory size in the node: 8402.23mb
[2021-06-01:18:51:35:INFO] Determined delimiter of CSV input is ','
[18:51:35] S3DistributionType set as FullyReplicated
[18:51:35] 73999x2 matrix with 147998 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[18:51:35] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=

In [16]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

-------------!

In [17]:
from sagemaker.serializers import CSVSerializer

test_data_array = test_data.drop(['frp'], axis=1).values #load the data into an array
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',')*1000 # and turn the prediction into an array
print(predictions_array.shape)

(31714,)


In [18]:
cm = pd.crosstab(index=test_data['frp'],columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 88.9%

Predicted      No Purchase    Purchase
Observed
No Purchase    100% (16)   100% (2)
Purchase        0% (0)      0% (0) 

